In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
DATASET_PATH = "/kaggle/input/face-mask-detection"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224,224),
    batch_size=32,
    class_mode='sparse',   
    subset='training',
    shuffle=True,
    seed=42
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224,224),
    batch_size=32,
    class_mode='sparse',   
    subset='validation',
    shuffle=False,
    seed=42
)

In [ ]:
base_model = MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
base_model.trainable = False 

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=outputs)

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',  # loss untuk sparse label integer
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
EPOCHS = 5

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.show()

In [ ]:
val_generator.reset()
Y_pred = model.predict(val_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print('Classification Report:')
print(classification_report(val_generator.classes, y_pred, 
                          target_names=list(train_generator.class_indices.keys())))

In [ ]:
cm = confusion_matrix(val_generator.classes, y_pred)
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=train_generator.class_indices.keys(),
            yticklabels=train_generator.class_indices.keys(),
            cmap='Blues')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random

In [ ]:
def load_saved_model(model_path):
    """Memuat model yang sudah disimpan"""
    try:
        loaded_model = load_model(model_path)
        print(f"Model berhasil dimuat dari: {model_path}")
        return loaded_model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

model_path = "/kaggle/working/face_mask_model.h5"
loaded_model = load_saved_model(model_path)

In [ ]:
class_names = ['mask_weared_incorrect', 'with_mask', 'without_mask']
print(f"Class names: {class_names}")

In [ ]:
def predict_single_image(model, image_path, class_names, target_size=(224, 224)):
    """Predict single image"""
    try:
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

In [ ]:
        prediction = model.predict(img_array, verbose=0)
        predicted_class = np.argmax(prediction[0])
        confidence = np.max(prediction[0])

In [ ]:
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.axis('off')

In [ ]:
        color = 'green' if confidence > 0.8 else 'orange' if confidence > 0.6 else 'red'

In [ ]:
        plt.title(f'Predicted: {class_names[predicted_class]}\nConfidence: {confidence:.4f}', 
                 fontsize=14, color=color, pad=20)
        plt.show()

In [ ]:
        print(f"📷 Image: {os.path.basename(image_path)}")
        print(f"🎯 Predicted: {class_names[predicted_class]}")
        print(f"📊 Confidence: {confidence:.4f}")

In [ ]:
        print("\n📈 All probabilities:")
        for i, (class_name, prob) in enumerate(zip(class_names, prediction[0])):
            if i == predicted_class:
                print(f"  ✅ {class_name}: {prob:.4f} ← PREDICTED")
            else:
                print(f"     {class_name}: {prob:.4f}")
        
        print("=" * 60)
        
        return predicted_class, confidence
    except Exception as e:
        print(f"❌ Error processing image {image_path}: {e}")
        return None, None

In [ ]:
def find_test_images(dataset_path, num_images=6):
    """Cari gambar acak untuk testing"""
    all_images = []

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                all_images.append(os.path.join(root, file))

    if len(all_images) > num_images:
        test_images = random.sample(all_images, num_images)
    else:
        test_images = all_images
    
    print(f"🔍 Found {len(all_images)} images total")
    print(f"🎲 Selected {len(test_images)} random images for testing")
    
    return test_images

In [ ]:
def test_multiple_images(model, dataset_path, class_names, num_test_images=6):
    """Test model dengan beberapa gambar acak"""
    print("\n" + "="*70)
    print("🚀 STARTING IMAGE TESTING")
    print("="*70)
    
    test_images = find_test_images(dataset_path, num_test_images)
    
    if not test_images:
        print("❌ No images found for testing!")
        return
    
    results = []

    for i, image_path in enumerate(test_images, 1):
        print(f"\n📋 Test {i}/{len(test_images)}")
        predicted_class, confidence = predict_single_image(model, image_path, class_names)
        
        if predicted_class is not None:
            results.append({
                'image_path': image_path,
                'predicted_class': predicted_class,
                'confidence': confidence
            })

    print("\n" + "="*70)
    print("📊 TESTING SUMMARY")
    print("="*70)
    
    if results:
        avg_confidence = np.mean([r['confidence'] for r in results])
        print(f"✅ Total tests: {len(results)}")
        print(f"📈 Average confidence: {avg_confidence:.4f}")

        class_counts = {}
        for class_name in class_names:
            class_counts[class_name] = 0
        
        for result in results:
            class_name = class_names[result['predicted_class']]
            class_counts[class_name] += 1
        
        print("\n🎯 Predictions distribution:")
        for class_name, count in class_counts.items():
            print(f"   {class_name}: {count} images")
    
    return results

In [ ]:
def print_model_info(model):
    """Print informasi model"""
    print("\n" + "="*50)
    print("🤖 MODEL INFORMATION")
    print("="*50)
    print(f"📐 Input shape: {model.input_shape}")
    print(f"📊 Output shape: {model.output_shape}")
    print(f"🔢 Number of classes: {len(class_names)}")
    print(f"📋 Classes: {class_names}")

In [ ]:
if loaded_model is not None:
    # Print model info
    print_model_info(loaded_model)
    
    # Test dengan gambar acak
    dataset_path = "/kaggle/input/face-mask-detection"
    results = test_multiple_images(loaded_model, dataset_path, class_names, num_test_images=6)
    
    print("\n🎉 Testing completed! Check the images and predictions above.")
else:
    print("❌ Model tidak berhasil dimuat!")

In [ ]:
def test_specific_image(model, image_path, class_names):
    """Test dengan image spesifik jika ada"""
    if os.path.exists(image_path):
        print(f"\n🔍 Testing specific image: {image_path}")
        predict_single_image(model, image_path, class_names)
    else:
        print(f"❌ Image not found: {image_path}")